### **环境特征处理_1**

#### **总览**

`OrganProcess`负责把建筑，兵线，子弹，BUFF，事件相关的信息转成一个向量传入网络模型

```python
def process_vec_organ(self, frame_state):
    self.generate_organ_info_dict(frame_state)      # 切分己方/敌方的防御建筑
    self.generate_hero_info_list(frame_state)       # 记录英雄（用于相对位置、视角镜像等）

    vec = []
    # 1) 己/敌防御塔（organ）特征
    vec += self.generate_one_type_organ_feature(self.main_camp_organ_dict, "ally_camp")
    vec += self.generate_one_type_organ_feature(self.enemy_camp_organ_dict, "enemy_camp")
    # 2) 小兵特征
    vec += self._encode_minions(frame_state)
    # 3) 子弹/弹道特征
    vec += self._encode_bullets(frame_state)
    # 4) 英雄BUFF摘要（己/敌）
    vec += self._encode_buff_summary(frame_state)
    # 5) 事件/草丛交战信号
    vec += self._encode_events_and_grass(frame_state)
    return vec

```

目前总的维度：
- 防御塔：敌我各7维
- 小兵信息：`5 + 5*K + 5*K = 45`
- 子弹信息：`2 + 4*K = 18`
- BUFF摘要：6
- 事件/草丛：3

#### **视角处理**

- 使用镜像视角，通过`self.transform_camp2_to_camp1 = (camp == "PLAYERCAMP_2")`进行阵营视角镜像
- 内部使用`view_dist=15000`简单判断是否在视野内
- 归一化策略


#### **防御塔**

对比初始仅加入了敌我防御塔完善信息，其余7维没有变化

#### **小兵特征**

输出结构：

- 汇总 5 维：`[a_cnt_norm, e_cnt_norm, a_hp_sum_norm, e_hp_sum_norm, push_depth]`
- 己方前K=4个小兵：`[alive, dx_norm, dz_norm, hp_ratio, atk_tower_flag]`
- 敌方同样

```python
    def _encode_minions(self, frame_state):
        """
        输出：5 + k*5 + k*5
        汇总(5)：[a_cnt_norm, e_cnt_norm, a_hp_sum_norm, e_hp_sum_norm, push_depth_norm]
        TopK单槽(5)：[alive, dx_norm, dz_norm, hp_ratio, atk_tower_flag]
        """
        npc = frame_state.get("npc_states", []) or []
        A = [u for u in npc if u.get("camp") == self.main_camp and u.get("sub_type") == "ACTOR_SUB_SOLDIER" and u.get("hp",0)>0]
        E = [u for u in npc if u.get("camp") != self.main_camp and u.get("sub_type") == "ACTOR_SUB_SOLDIER" and u.get("hp",0)>0]

        def _hp_sum(lst): return float(sum(max(0.0, float(u.get("hp",0.0))) for u in lst))
        a_cnt = min(len(A), 10) / 10.0
        e_cnt = min(len(E), 10) / 10.0
        a_hpsum = min(_hp_sum(A), 20000.0) / 20000.0
        e_hpsum = min(_hp_sum(E), 20000.0) / 20000.0

        def _min_dist_to(target, lst):
            if not lst or not target: return self.RANGE_NORM
            tx, tz = self._pos(target); res = self.RANGE_NORM
            for u in lst:
                ux, uz = self._pos(u)
                d = math.hypot(ux-tx, uz-tz)
                if d < res: res = d
            return min(res, self.RANGE_NORM)
        ally_tower  = self.main_camp_organ_dict.get("tower")
        enemy_tower = self.enemy_camp_organ_dict.get("tower")
        a_front = _min_dist_to(enemy_tower, A)
        e_front = _min_dist_to(ally_tower, E)
        push_depth = (e_front - a_front) / self.RANGE_NORM  # [-1,1]
        me = self.main_hero_info["actor_state"]
        mx, mz = self._pos(me)
        def _dx_norm(x):
            if self.transform_camp2_to_camp1: x = -x
            return (x + 15000.0) / 30000.0
        def _slot(u, target_tower_id):
            if not u:
                return [0.0]*5
            ux, uz = self._pos(u)
            dx = ux - mx; dz = uz - mz
            hp = float(u.get("hp",0.0)); mxhp = float(u.get("max_hp",1.0))
            hp_ratio = 0.0 if mxhp <= 0 else (hp/mxhp)
            atk_tower_flag = 1.0 if int(u.get("attack_target", -1)) == int((target_tower_id or -2)) else 0.0
            return [1.0, _dx_norm(dx), _dx_norm(dz), hp_ratio, atk_tower_flag]
        def _topk(lst):
            return sorted(lst, key=lambda u: math.hypot(self._pos(u)[0]-mx, self._pos(u)[1]-mz))[:self.MINION_TOPK]

        ally_slots  = sum((_slot(u, (enemy_tower or {}).get("runtime_id")) for u in _topk(A)), [])
        enemy_slots = sum((_slot(u, (ally_tower  or {}).get("runtime_id")) for u in _topk(E)), [])
        while len(ally_slots)  < self.MINION_TOPK*5: ally_slots += [0.0]*5
        while len(enemy_slots) < self.MINION_TOPK*5: enemy_slots += [0.0]*5

        return [a_cnt, e_cnt, a_hpsum, e_hpsum, push_depth] + ally_slots + enemy_slots
    
```

#### **子弹特征**

输出结构：

- 汇总2维：`[bullet_cnt_norm, incoming_cnt_norm]`
- 朝向我方的弹药topk=4，每弹4维`[dx_norm, dz_norm, dist_norm, cos_to_me]`

```python
    def _encode_bullets(self, frame_state):
        """
        输出：2 + K*4 维
        汇总(2)：[bullet_cnt_norm, incoming_cnt_norm]
        TopK来弹单槽(4)：[dx_norm, dz_norm, dist_norm, cos_to_me]
        """
        me = self.main_hero_info["actor_state"]
        mx, mz = self._pos(me)
        bullets = frame_state.get("bullets", []) or []
        enemy_bullets = [b for b in bullets if b.get("camp") == self.main_camp]

        def _cos_to_me(b):
            bx, bz = self._pos(b); v = b.get("use_dir") or {}
            vx, vz = float(v.get("x",0.0)), float(v.get("z",0.0))
            to_me_x, to_me_z = (mx - bx), (mz - bz)
            num = vx*to_me_x + vz*to_me_z
            den = math.hypot(vx, vz) * math.hypot(to_me_x, to_me_z)
            return 0.0 if den <= 1e-6 else max(-1.0, min(1.0, num/den))
        incoming = [b for b in enemy_bullets if _cos_to_me(b) > 0.0]

        cnt_norm = min(len(enemy_bullets), 20) / 20.0
        incoming_cnt_norm = min(len(incoming), 10) / 10.0

        def _dx_norm(x):
            if self.transform_camp2_to_camp1: x = -x
            return (x + 15000.0) / 30000.0
        def _score(b):
            bx, bz = self._pos(b)
            dist = math.hypot(bx-mx, bz-mz)
            return (-dist, _cos_to_me(b))
        incoming_sorted = sorted(incoming, key=_score)[:self.BULLET_TOPK]

        slots = []
        for b in incoming_sorted:
            bx, bz = self._pos(b)
            dx = _dx_norm(bx - mx); dz = _dx_norm(bz - mz)
            dist_norm = min(math.hypot(bx-mx, bz-mz), self.RANGE_NORM)/self.RANGE_NORM
            slots += [dx, dz, dist_norm, _cos_to_me(b)]
        while len(slots) < self.BULLET_TOPK*4: slots += [0.0]*4

        return [cnt_norm, incoming_cnt_norm] + slots
```

#### **BUFF**

汇总：对己/敌英雄各输出3维

- `buff_mark_cnt_norm = min(印记数量,10)/10`
- `buff_mark_layers_norm = min(印记层数之和,10)/10`
- `buff_skill_group_cnt_norm = min(生效 BUFF 组数,10)/10`

```python
    def _encode_buff_summary(self, frame_state):
        """
        每英雄： [buff_mark_cnt_norm, buff_mark_layers_norm, buff_skill_group_cnt_norm]
        总计 6 维
        """
        # 敌方阵营（内联，不引入新函数）
        if self.main_camp == "PLAYERCAMP_1":
            enemy_camp = "PLAYERCAMP_2"
        elif self.main_camp == "PLAYERCAMP_2":
            enemy_camp = "PLAYERCAMP_1"
        elif isinstance(self.main_camp, int) and self.main_camp in (0, 1):
            enemy_camp = 1 - self.main_camp
        elif isinstance(self.main_camp, int) and self.main_camp in (1, 2):
            enemy_camp = 3 - self.main_camp
        else:
            enemy_camp = "PLAYERCAMP_2"

        def _hero(camp_value):
            for h in frame_state.get("hero_states", []) or []:
                if (h.get("actor_state") or {}).get("camp") == camp_value:
                    return h
            return None

        def _buff_vec(h):
            if not h:
                return [0.0, 0.0, 0.0]
            bs = h.get("buff_state") or (h.get("actor_state") or {}).get("buff_state") or {}
            marks = bs.get("buff_marks") or []
            skills = bs.get("buff_skills") or []
            cnt = min(len(marks), 10) / 10.0
            layer_sum = sum(int(m.get("layer", 0)) for m in marks)
            layer_norm = min(layer_sum, 10) / 10.0
            skill_cnt_norm = min(len(skills), 10) / 10.0
            return [cnt, layer_norm, skill_cnt_norm]

        ally = _buff_vec(_hero(self.main_camp))
        enemy = _buff_vec(_hero(enemy_camp))
        return ally + enemy
```

#### **草丛**

给出`grass_engage`，当我在草，敌不在草时给1，其余给0

#### **详细描述**

可以参考[详细说明](https://chatgpt.com/s/t_68aee0e0ece48191ad28783832ebf587)
